# Práctica: Análisis de datos para la exploración de pozos

## Resumen

El objetivo de este ejercicio es, a través de un análisis de datos, determinar en qué pozos debe de enfocarse la atención para la extracción de petróleo.

La metodología adoptada en este análisis consiste en analizar la serie de tiempo para cada uno de los pozos con datos para 14 meses. Es decir, sólo se tomará en cuenta aquellos pozos para los cuales existe información para el periodo comprendido entre enero de 2016 y febrero de 2017, ya que dicho rango es el máximo posible de acuerdo con la base de datos que fue proporcionada.

Para cada uno de los pozos que cumplió con el criterio anterior se ajustó una regresión lineal cuya variable dependiente $y$ es la producción de petróleo, y la variable independiente $x$ es el índice del periodo observado. Debido a que se cuenta con 14 meses, el índice de cada periodo es un número entero entre 0 y 13 ( $x$ E [0,14]). Cada una de estas regresiones se ajustó para fines meramente descriptivos, pues su propósito es obtener una línea de tendencia durante los 14 periodos analizados. Para conocer si la tendencia es positiva o negativa se observó el valor de la pendiente resultante del cómputo de una función lineal a través del método de mínimos cuadrados ordinarios; para conocer si dicha tendencia, o comportamiento, ha sido estable a lo largo del periodo analizado se observó el coeficiente de determinación, o $R^2$ ($R^2$ e (0,1)).

Los pozos seleccionados fueron aquellos cuya línea de tendencia fue positiva y con una bondad de ajuste por arriba del 60 por ciento.


## Introducción

Esta práctica se desarrolló con el lenguaje de programación Python y este documento se hizo con una herramienta llamada Jupyter.

Este trabajo está dividido en "n" secciones...

Los archivos con los cuales se hizo este análisis se encuentran albergados en un repositorio público en Github. Éste puede descargarse accediendo a la siguiente liga: https://github.com/danielurencio/pozos.

$x = a + b*x$

## Preprocesamiento de datos

### Transformación de formato

Para llevar a cabo este análisis se obtuvo un archivo en formato Excel (xlsx) el cual contiene 138 mil 876 filas y 13 columnas. Antes de cargar la información a una terminal de línea de comando de Python se transformó el archivo original a un formato delimitado por comas (csv). Posteriormente, se eliminaron las primeras 8 líneas de dicha transformación y se sustituyeron algunos caracteres para evitar problemas al interpretar algunos caracteres latinos.

Se creó, por lo tanto, un archivo para ser ejecutado con UNIX Shell, éste contiene 14 líneas. En términos generales, los comandos de este archivo, que se enlistan a continuación, crean y modifican el archivo delimitado por comas (pozos.csv) que será utilizado en Python para llevar a cabo todo el análisis.

In [28]:
# xlsx2csv P*.xlsx > pozos.csv    # Convertir a CSV.
# sed -i '1,8d' pozos.csv         # Eliminar las primeras ocho líneas.
# sed -i "s/  */ /g" pozos.csv    # Borrar dobles espacios.
# sed -i "s/Á/A/g" pozos.csv
# sed -i "s/á/a/g" pozos.csv
# sed -i "s/É/E/g" pozos.csv
# sed -i "s/é/e/g" pozos.csv
# sed -i "s/Í/I/g" pozos.csv
# sed -i "s/í/i/g" pozos.csv
# sed -i "s/Ó/O/g" pozos.csv
# sed -i "s/ó/o/g" pozos.csv
# sed -i "s/Ú/U/g" pozos.csv
# sed -i "s/Ñ/N/g" pozos.csv
# sed -i "s/ñ/n/g" pozos.csv

### Dependencias y procesamiento adicional en Python

Las dependencias que se utilizaron para este análisis se enlistan a continuación. (Todas las líneas que se muestran abajo, con excepción de la primera, se encuentran en el archivo original del repositorio. El comando "%matplotlib inline" se incluye para poder visualizar gráficas dentro de este documento; dicho comando es, por lo tanto, una funcionalidad exclusiva de Jupyter).

In [29]:
%matplotlib inline
import pandas
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats

Una vez que se cuenta con las dependencias es posible cargar el archivo "pozos.csv". Para ello se utilizará la librería "Pandas" y se declarará una variable con nombre "data" para almacenar en ésta la información del archivo. Asimismo, se cambiará el nombre de todas las columnas para evitar problemas con caracteres latinos (principalmente con palabras acentuadas).

Para conocer el número de pozos puede hacerse lo siguiente.

In [30]:
## Crear DataFrame con los datos sobre pozos.
data = pandas.read_csv("pozos.csv",encoding="utf-8",delimiter=",");

## Renombrar columnas.
data.columns = ["periodo","asignacion","region_fiscal","campo_oficial","pozo","ronda_licitacion",
                "compania","produccion_petroleo(mbd)","produccion_gas_asociado(mmpcd)",
                "produccion_gas_no_asociado(mmpcd)","produccion_total_de_gas(mmpcd)","tipo_de_aceite",
                "pozo_productor"];

Ahora que la información del archivo se encuentra en la línea de comando es posible visualizar ésta como una tabla (DataFrame).

In [31]:
data.head()

,periodo,asignacion,region_fiscal,campo_oficial,pozo,ronda_licitacion,compania,produccion_petroleo(mbd),produccion_gas_asociado(mmpcd),produccion_gas_no_asociado(mmpcd),produccion_total_de_gas(mmpcd),tipo_de_aceite,pozo_productor
0,Jan-16,A-0001 - Campo Abkatun,Aguas Someras,Abkatun,ABKATUN-211--,R0,Pemex,0.756978,1.923678,0.0,1.923678,Ligero,1
1,Jan-16,A-0001 - Campo Abkatun,Aguas Someras,Abkatun,ABKATUN-216--,R0,Pemex,0.073139,0.051653,0.0,0.051653,Ligero,1
2,Jan-16,A-0001 - Campo Abkatun,Aguas Someras,Abkatun,ABKATUN-221--,R0,Pemex,0.584042,1.137749,0.0,1.137749,Ligero,1
3,Jan-16,A-0001 - Campo Abkatun,Aguas Someras,Abkatun,ABKATUN-223--,R0,Pemex,0.209442,1.044266,0.0,1.044266,Ligero,1
4,Jan-16,A-0001 - Campo Abkatun,Aguas Someras,Abkatun,ABKATUN-35--,R0,Pemex,1.003177,1.054900,0.0,1.054900,Ligero,1


In [32]:
## Pozos (sin repetir valores) 
pozos = np.unique(data.pozo.values);
print len(pozos)
print pozos

11646
[u'5 PRESIDENTES-12--' u'5 PRESIDENTES-145D-' u'5 PRESIDENTES-151--' ...,
 u'ZAPOTALILLO-3--' u'ZAPOTALILLO-8--' u'ZAPOTALILLO-9--']


Lo anterior indica que existe información para 11 mil 646 pozos.

Dado que este análisis toma en cuenta la serie de tiempo para cada pozo, es necesario asegurarse de que los datos están ordenados temporalmente. Por ello, como medida de precaución, se transformarán los campos que incluyen las fechas a un formato temporal. Hasta este punto, la información sobre el periodo de tiempo de cada pozo está almacenada como un conjunto de caracteres, por lo que si se pretende ordenar ésta de modo ascendente se obtendrá un orden alfabético y no temporal.

In [35]:
# Ordenar el campo con periodos de modo ascendente.
data.sort_values(by="periodo").head()["periodo"]

0       2016-01-01 00:00:00
4627    2016-01-01 00:00:00
1       2016-01-01 00:00:00
2       2016-01-01 00:00:00
3       2016-01-01 00:00:00
Name: periodo, dtype: object

Para corregir lo anterior se cambiará el formato del campo con los periodos para que en el código su información sea interpretada como fechas y no como caracteres.

In [34]:
periodos = data.periodo.values;

for i in range(len(periodos)):
    periodos[i] = periodos[i].encode("utf-8");
    periodos[i] = datetime.strptime(periodos[i],"%b-%y");

periodos = pandas.DataFrame({ "periodos":periodos[:] })
data.periodos = periodos;

# Ordenar por fecha
data = data.sort_values(by="periodo")


In [23]:
#a.plot.density(); plt.show()